In [1]:
from functions.chat_bot_v2 import *
from functions.filomena_utils_v2 import *

2024-01-06 04:35:49.744 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [2]:
fil = Filomena()

In [3]:
import os 
files = [f'text_data/{file}' for file in os.listdir('text_data')]

In [4]:
fil.initialize(files= files)

In [5]:
fil.core_piece.get_instruction('I want to have a restaurant recommendation please', [])

'[INSTRUCTION: Prepare Restaurant Recommendation] | query `CONTEXT`: {\'[INSTRUCTION: Identification]\': {\'description\': \'The assistant should greet the user.\', \'when to use\': \'When the user greets the assistant and the content of the last message from the assistant is empty.\'}, \'[INSTRUCTION: Question]\': {\'description\': \'The assistant should answer the question of the user if they can.\', \'when to use\': \'When the user asks non-restaurant-related questions (e.g. a question about a person, about FlavourFlix, about the virtual assistant or about portuguese gastronomic culture).\'}, \'[INSTRUCTION: Restaurant Description]\': {\'description\': \'The assistant should provide with a description of the restaurant in the `query`.\', \'when to use\': \'The user inquires about a specific restaurant, providing atleast the name of the restaurant. CAUTION: "The Adventurer", "Fine Dining Connoiser", "Comfort Food Lover", "Low Cost Foodie" and "Conscious Eater" are not restaurant name

In [6]:
fil.generate_response('I want to have a restaurant recommendation please')

'Where would you like to find a restaurant?'

In [11]:
fil.generate_response('albufeira')

"'A Severa - Fado' is a Portuguese restaurant located in Lisbon, known for its homemade cuisine and average price of 56. It is open from 20:00 to 01:00 on most days, except for Wednesdays when it is closed."

In [7]:
fil.generate_response('Tell me about Portugalia')

"The restaurant 'Portugália - Cais do Sodré' is a seafood restaurant located in Lisboa. The average price is 20 units. The restaurant is open from Monday to Sunday, from 12:00 to 22:30."

In [7]:
fil.generate_response('Tell me about Galina da vizinha')

In [6]:
def personality_based_recommendation(personality: str, location:str=None):
    """Gets restaurant recommendations based on the user's personality and loation if provided.
    Parameters:
        - personality (str): The user's personality. Can only be one of the following: 
                - [The Adventurer, Fine Dining Connoisseur, Low Cost Foodie, Conscious Eater, Comfort Food Lover].
        - location (str): The user's location.
    Returns:
        - rec_restaurants (DataFrame): A DataFrame with the recommended restaurants.
    """

    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')

    #If we have access to the location preferece, we will filter the restaurants based on the location and nearby locations
    if location is not None:
        #Find the restaurants in the location
        equal_location = restaurant_data[restaurant_data['location'] == location]
        rec_restaurants = equal_location
        #Find the restaurants
        latitude, longitude = find_coordinates(location)
        # if latitude is not None and longitude is not None:
        #     selected_location = Location(latitude, longitude)
        #     restaurant_data = nearYou(selected_location, restaurant_data)
        #     restaurant_data['minutes_away_car'] = restaurant_data.apply(lambda row: selected_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
        #     near_location = restaurant_data[restaurant_data['minutes_away_car'] <= 20]
        #     rec_restaurants = pd.concat([near_location, equal_location], ignore_index=True)
        # else:
        #     rec_restaurants = equal_location
    else:
        rec_restaurants = restaurant_data            
        
    #Now we will filter the restaurants based on the personality
    if personality == 'The Adventurer':
        rec_restaurants = rec_restaurants[rec_restaurants['cuisine'].isin(['Peruvian', 'Lebanese', 'Iranian', 'Vietnamese', 'Fusion', 'Nepalese', 'Thai', 'Asian',
                        'Korean', 'Tibetan', 'Chinese', 'African', 'Syrian', 'Venezuelan', 'Indian', 'Japanese','South American',
                        'Argentinian', 'Mexican'])]
        rec_restaurants = pd.concat([rec_restaurants[rec_restaurants['style'] == 'Ethnic'], rec_restaurants[rec_restaurants['style'] != 'Ethnic']])
        rec_restaurants.sort_values(by=['foodRatingSummary'], ascending=False, inplace=True)

    
    elif personality == 'Fine Dining Connoisseur':
        rec_restaurants = rec_restaurants[rec_restaurants['style'].isin(['Fine Dining', 'Modern', 'View', 'Central Location', 'Brunch', 'Meetings'])] 
        rec_restaurants = rec_restaurants[(rec_restaurants['ambienceRating'] >= 8) & (rec_restaurants['serviceRating'] >= 8)]
        rec_restaurants.sort_values(by=['ambienceRatingSummary', 'serviceRatingSummary'], ascending=False, inplace=True)
    
    elif personality == 'Low Cost Foodie':
        #lowcost_rest = ['Street Food', 'Buffet', 'Brazilian', 'Not Available', 'Homemade', 'Family', 'Healthy',]
        #rec_restaurants = rec_restaurants[rec_restaurants['style'].isin(lowcost_rest) & (restaurant_data['averagePrice'] < 12) & (restaurant_data['promotions'] =! 'No Offers')]
        rec_restaurants = rec_restaurants[(rec_restaurants['promotions'] != 'No Offers')]
        rec_restaurants.sort_values(by=['averagePrice', 'foodRatingSummary'], ascending=[True, False], inplace=True)

    elif personality == 'Conscious Eater':
        conscious_cuisine = ['Mediterranean', 'Greek', 'Vegan', 'Vegetarian', 'Traditional']
        rec_restaurants = rec_restaurants[rec_restaurants['cuisine'].isin(conscious_cuisine)]
        rec_restaurants = pd.concat([rec_restaurants[(rec_restaurants['style'] == 'Healthy') | (rec_restaurants['style'] == 'Brunch')], 
                                     rec_restaurants[(rec_restaurants['style'] != 'Healthy') | (rec_restaurants['style'] != 'Brunch')]])
        rec_restaurants.sort_values(by=['foodRatingSummary'], ascending=False, inplace=True)
    
    if personality == 'Comfort Food Lover':
        comfort_cuisine = ['Traditional', 'American', 'Portuguese', 'Brazilian', 'Pizzeria', 'Mexican',
                            'Pub grub', 'Grilled', 'Spanish', 'Mediterranean', 'Italian', 'Meat', 'Steakhouse']
        rec_restaurants = rec_restaurants[rec_restaurants['cuisine'].isin(conscious_cuisine)]
        rec_restaurants = pd.concat([rec_restaurants[(rec_restaurants['style'] == 'Homemade') | (rec_restaurants['style'] == 'Healthy')], 
                                     rec_restaurants[(rec_restaurants['style'] != 'Homemade') | (rec_restaurants['style'] != 'Healthy')]])
        rec_restaurants.sort_values(by=['foodRatingSummary'], ascending=False, inplace=True)

    return rec_restaurants.head(5)

In [7]:


def user_preferences_recommendation(location: str=None, nationality: str= None, cuisine_type: str=None,
                                     restaurant_style: str = None, price_range: str=None, dinner_hour: str=None, 
                                     lunch_hour: str=None, favourite_food: str=None, preference: str='ratingValue'):
    
    """Gets restaurant recommendations based on the user's preferences.
    Parameters:
        - location (str):  The city the user wants to eat in.
        - nationality (str): Nationality of the food the user wants to eat.
        - cuisine_type (str): The type of cuisine the user wants to eat.
        - restaurant_style (str): The style of the restaurant the user wants to eat at.
        - price_range (str): The price value in euros the user is willing to pay per meal per person.
        - dinner_hour (str): The timeslot the user wants to have dinner in the format "HH:MM - HH:MM".
        - lunch_hour (str): The timeslot the user wants to have lunch in the format "HH:MM - HH:MM".
        - favourite_food (str): The specific dish or meal the user wants to eat.
        - preference (str): The user's preference. Can only be one of the following:
                - [ratingValue, averagePrice, ambienceRatingSummary, serviceRatingSummary, foodRatingSummary].
    Returns:
        - rec_restaurants (DataFrame): A DataFrame with the recommended restaurants.

    """
    
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')

    #If we have access to the location preferece, we will filter the restaurants based on the location and nearby locations
    if location is not None:
        #Find the restaurants in the location
        equal_location = restaurant_data[restaurant_data['location'] == location]
        rec_restaurants = equal_location
        #Find the restaurants
        latitude, longitude = find_coordinates(location)
        # if latitude is not None and longitude is not None:
        #     selected_location = Location(latitude, longitude)
        #     restaurant_data = nearYou(selected_location, restaurant_data)
        #     restaurant_data['minutes_away_car'] = restaurant_data.apply(lambda row: selected_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
        #     near_location = restaurant_data[restaurant_data['minutes_away_car'] <= 20]
        #     rec_restaurants = pd.concat([near_location, equal_location], ignore_index=True)
        # else:
        #     rec_restaurants = equal_location
    else:
        rec_restaurants = restaurant_data  

    if len(rec_restaurants) > 5:
        #Now we will filter the restaurants based on the user preferences
        if cuisine_type is not None and nationality is not None:
            cuisine_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
            nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
            rec_restaurants = rec_restaurants[(rec_restaurants['cuisine'] == cuisine_match) | (rec_restaurants['cuisine'] == nationality_match)]
        elif cuisine_type is not None:
            cuisine_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
            rec_restaurants = rec_restaurants[(rec_restaurants['cuisine'] == cuisine_match) ]
        elif nationality is not None:
            nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
            rec_restaurants = rec_restaurants[(rec_restaurants['cuisine'] == nationality_match)]
    else:
        return rec_restaurants
    
    if len(rec_restaurants) > 5:
        if restaurant_style is not None:
            style_match = get_data_match(restaurant_data, restaurant_style, 'style')
            rec_restaurants = rec_restaurants[(rec_restaurants['style'] == style_match)]
    else:
        return rec_restaurants

    if len(rec_restaurants) > 5:
        if favourite_food is not None:
            if detect(favourite_food) == 'en':
                rec_restaurants[rec_restaurants['menu_en'].str.contains(favourite_food)]
            else:
                rec_restaurants[rec_restaurants['menu_pre_proc'].str.contains(favourite_food)]
        else:
            rec_restaurants = rec_restaurants
    else:
        return rec_restaurants
    
    if len(rec_restaurants) > 5:
        if price_range is not None:
            rec_restaurants[rec_restaurants['averagePrice'] <= price_range +4]
    else:
        return rec_restaurants

    if len(rec_restaurants) > 5:
        if lunch_hour is not None and dinner_hour is not None:
            rec_restaurants = filter_schedule(rec_restaurants, dinner_hour, lunch_hour)
        elif dinner_hour is not None:
            rec_restaurants = filter_schedule(rec_restaurants, dinner_hour, lunch_hour)
        elif lunch_hour is not None:
            rec_restaurants = filter_schedule(rec_restaurants, dinner_hour, lunch_hour)
    else:
        return rec_restaurants
    
    if preference == 'averagePrice':
        rec_restaurants = rec_restaurants.sort_values(by=['averagePrice'], ascending=True)
    else:
        rec_restaurants = rec_restaurants.sort_values(by=[preference], ascending=False)

    return rec_restaurants.head(5)
    


In [8]:
def get_recommendation(personality: str=None, location: str =None, nationality: str= None, cuisine_type: str=None,
                                     restaurant_style: str = None, price_range: str=None, dinner_hour: str=None, 
                                     lunch_hour: str=None, favourite_food: str=None, preference: str='ratingValue'):
    """Gets restaurant recommendations based on the user's personality or 
    personal preferences stated throughout the chat.
    Parameters:
        - personality (str): The user's personality. Can only be one of the following:
                - [The Adventurer, Fine Dining Connoisseur, Low Cost Foodie, Conscious Eater, Comfort Food Lover].
        - location (str):  The city the user wants to eat in.
        - nationality (str): Nationality of the food the user wants to eat.
        - cuisine_type (str): The type of cuisine the user wants to eat.
        - restaurant_style (str): The style of the restaurant the user wants to eat at.
        - price_range (str): The price value in euros the user is willing to pay per meal per person.
        - dinner_hour (str): The timeslot the user wants to have dinner in the format "HH:MM - HH:MM".
        - lunch_hour (str): The timeslot the user wants to have lunch in the format "HH:MM - HH:MM".
        - favourite_food (str): The specific dish or meal the user wants to eat.
        - preference (str): The user's preference. Can only be one of the following:
                - [ratingValue, averagePrice, ambienceRatingSummary, serviceRatingSummary, foodRatingSummary]. 
    Returns:
        - rec_restaurants (DataFrame): A DataFrame with the recommended restaurants.
    """
    if personality != None:
        recommendations = personality_based_recommendation(personality, location)
    else:
        recommendations = user_preferences_recommendation(location, nationality, cuisine_type,restaurant_style, price_range, dinner_hour, 
                                     lunch_hour, favourite_food, preference)
    return recommendations

In [9]:
get_recommendation(personality='The Adventurer', location='Lisbon')

,url,name,address,photo,averagePrice,chefName1,chefName2,chefName3,cuisine,michelin,...,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,outdoor_area,current_occupation,menu_pre_proc,menu_en,menu_pt
731,https://www.thefork.com/restaurant/seisan-r728776,Seisan,"R. da Madalena 271,Lisbon, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,30,Not Applicable,Not Applicable,Not Applicable,Japanese,0,...,Almada,9.5,9.8,9.7,"{'Visa Electron', 'Visa'}",0,0,{'Maki Monos 6 Peças Hossomaki': {'Kappa Maki'...,{'Maki Monos 6 Hossomaki Pieces': {'Kappa Maki...,{'Maki Monos 6 Peças Hossomaki': {'Kappa Maki'...
711,https://www.thefork.com/restaurant/katanasushi...,KatanaSushi (Santos),"R. Presidente Arriaga nº 55,Lisbon, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,17,Not Applicable,Not Applicable,Not Applicable,Japanese,0,...,Almada,9.2,9.7,9.5,"{'Visa Electron', 'Visa'}",0,41,"{'Duração média de refeição 1:30h': {}, 'Start...","{'Average meal duration 1:30h': {}, 'Starter':...","{'Duração média de refeição 1:30h': {}, 'Inici..."
757,https://www.thefork.com/restaurant/ajitama-ram...,Ajitama Ramen Bistro - Alecrim,"R. do Alecrim 47A, 1200-014 Lisboa, Portugal,L...",https://res.cloudinary.com/tf-lab/image/upload...,19,Not Applicable,Not Applicable,Not Applicable,Japanese,0,...,Almada,9.3,9.7,9.4,"{'Credit Card', 'Visa'}",0,16,{'Entradas': {'Padron togarashi': {'price': 5....,{'Appetizer': {'Padron togarashi': {'price': 5...,{'Starters': {'Padrão togarashi': {'price': 5....
827,https://www.thefork.com/restaurant/izanagi-sus...,IZANAGI SushiCafé,"Doca de Santo Amaro Armazém 0,Lisbon, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,20,Daniel Rente,Jose Barros,Francisco Correia,Japanese,0,...,Almada,9.4,9.6,9.3,"{'Credit Card', 'Visa'}",0,19,"{'Starter': {'Sopa Miso': {'price': 3.0, 'desc...","{'Starter': {'Miso soup': {'price': 3.0, 'desc...","{'Iniciante': {'Sopa de missô': {'price': 3.0,..."
865,https://www.thefork.com/restaurant/tentacoes-d...,Tentações de Gôa,"Rua S. Pedro Mártir - 23,Lisbon, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,14,Maria dos Anjos Martins,Not Applicable,Not Applicable,Indian,0,...,Almada,9.4,9.6,9.6,{'Cash'},0,0,{'Entradas': {'Bojés com Chetnim': {'price': 2...,{'Appetizer': {'Crazy like Chetnim': {'price':...,{'Starters': {'Louco como Chetnim': {'price': ...
